# Installs

In [1]:
%pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117 -q

Note: you may need to restart the kernel to use updated packages.



This may take a while

In [2]:
!pip install wandb --quiet
!pip install python-Levenshtein -q
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget -q
%cd ctcdecode
!pip install . -q
%cd ..

!pip install torchsummaryX -q

Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1102 (delta 16), reused 32 (delta 14), pack-reused 1063
Receiving objects: 100% (1102/1102), 782.27 KiB | 13.49 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/kaggle/working/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Counting objects: 100% (26/26), done.        
remote: Compressing objects: 100% (9/9), done.        
remote: Total 82 (delta 19), reused 17 (delta 17), pack-reused 56        
Receiving objects: 100% (82/82), 13.34 KiB | 3.34 MiB/s, done.
Resolving deltas: 100% (36/36), done.
Cloning into '/kaggle/working

# Imports

In [3]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Device:  cuda


# Kaggle Setup

In [4]:
# TODO: Use the same Kaggle code from HW1P2
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"danielkwizera","key":"858f28b48352b29cb72fa9f3815c1b5d"}')
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73256 sha256=1ae15095c7a7bef544b10449b0f6b0275a5b5f5c261f4301bc037aea8a759df0
  Stored in directory: /root/.cache/pip/wheels/0b/76/ca/e58f8afa83166a0e68f0d5cd2e7f99d260bdc40e35da080eee
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.16
    Uninstalling kaggle-1.5.16:
      Successfully uninstalled kaggle-1.5.16


In [5]:
!kaggle competitions download -c automatic-speech-recognition-asr

100%|██████████████████████████████████████▉| 3.97G/3.97G [00:25<00:00, 180MB/s]
100%|███████████████████████████████████████| 3.97G/3.97G [00:25<00:00, 164MB/s]


In [6]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-f23-hw3p2  automatic-speech-recognition-asr.zip	ctcdecode
'''
!unzip -q automatic-speech-recognition-asr.zip
!ls

11-785-f23-hw3p2  automatic-speech-recognition-asr.zip	ctcdecode


In [7]:
!pip install speechpy

In [8]:
import speechpy
import glob

# Google Drive

In [9]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# Dataset and Dataloader

In [10]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@",
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W",
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R",
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w",
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y",
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D",
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O",
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]

### Train Data

In [11]:
class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self,root_dir,partition,transforms,limit=None,train_460=True):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        # self.mfcc_dir = #TODO
        # self.transcript_dir = #TODO
        self.mfcc_dir = f"{root_dir}/{partition}*/mfcc/*" #TODO.     # '*' for regex
        self.transcript_dir = f"{root_dir}/{partition}*/transcript/*"#TODO


        # self.mfcc_files = #TODO
        # self.transcript_files = #TODO
        self.mfcc_files =sorted(glob.glob(self.mfcc_dir))
        self.transcript_files= sorted(glob.glob(self.transcript_dir))


        self.PHONEMES = PHONEMES

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        # self.length = NotImplemented
        assert len(self.mfcc_files) == len(self.transcript_files)

        self.mfccs = []
        self.transcripts = []

        for file in self.mfcc_files:
          self.mfccs.append(np.load(file))
        for file in self.transcript_files:
          ht = np.load(file)[1:-1]
          self.transcripts.append(np.array([self.PHONEMES.index(i) for i in ht]))

        print('Lenght before training =',len(self.mfccs))

        
        if train_460:
          partition_460 = 'train-clean-360'
          self.mfcc_files_460= sorted(glob.glob(f"{root_dir}/{partition_460}*/mfcc/*"))
          self.transcript_files_460= sorted(glob.glob(f"{root_dir}/{partition_460}*/transcript/*"))


          for file in self.mfcc_files_460:
            self.mfccs.append(np.load(file))
          for file in self.transcript_files_460:
            ht =np.load(file)[1:-1]
            self.transcripts.append(np.array([self.phonemes.index(i) for i in ht]))

        print('Length after training =',len(self.mfccs))

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfccs)

        assert len(self.mfccs) == len(self.transcripts)
        

        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''


    def __len__(self):

        '''
        TODO: What do we return here?
        '''
        # raise NotImplemented
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''

        # raise NotImplemented

        # mfcc = ___ # TODO
        # transcript = ___ # TODO
        # return mfcc, transcript
        mfcc = self.mfccs[ind]
        transcript = self.transcripts[ind]

        return torch.FloatTensor(mfcc), torch.LongTensor(transcript)


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features,
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = [i[0] for i in batch] # TODO
        # batch of output phonemes
        batch_transcript = [i[1] for i in batch] # TODO

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True) # TODO
        lengths_mfcc = [len(i) for i in batch_mfcc] # TODO

        batch_transcript_pad = pad_sequence(batch_transcript, batch_first = True) # TODO
        lengths_transcript = [len(i) for i  in batch_transcript] # TODO

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?

        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)

        # # batch of input mfcc coefficients
        # batch_mfcc = ___ # TODO
        # # batch of output phonemes
        # batch_transcript = ___ # TODO

        # # HINT: CHECK OUT -> pad_sequence (imported above)
        # # Also be sure to check the input format (batch_first)
        # batch_mfcc_pad = ___ # TODO
        # lengths_mfcc = ___ # TODO

        # batch_transcript_pad = ___ # TODO
        # lengths_transcript = ___ # TODO

        # # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        # #                  -> Would we apply transformation on the validation set as well?
        # #                  -> Is the order of axes / dimensions as expected for the transform functions?

        # # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        # return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)



### Test Data

In [12]:
# Test Dataloader
#TODO
class AudioDatasetTest(torch.utils.data.Dataset):

  def __init__(self,root_dir,partition,transforms,limit=None):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = f"{root_dir}/{partition}*/mfcc/*" #TODO.     # '*' for regex

        # self.mfcc_files = os.listdir(self.mfcc_dir) #TODO
        # self.transcript_files = os.listdir(self.mfcc_dir) #TODO

        self.mfcc_files =sorted(glob.glob(self.mfcc_dir))

        self.PHONEMES = PHONEMES

        self.mfccs = []

        for files in self.mfcc_files:
          self.mfccs.append(np.load(files))

        print('AUDIODATASET TEST LOADED ...')

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)

        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''


  def __len__(self):
     '''
     TODO: What do we return here?
     '''
     return self.length
     # raise NotImplemented

  def __getitem__(self, ind):
    '''
    TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

    If you didn't do the loading and processing of the data in __init__,
    do that here.

    Once done, return a tuple of features and labels.
    '''
    mfcc = self.mfccs[ind]

    return torch.FloatTensor(mfcc)
    # raise NotImplemented


  def collate_fn(self,batch):
      '''
      TODO:
      1.  Extract the features and labels from 'batch'
      2.  We will additionally need to pad both features and labels,
          look at pytorch's docs for pad_sequence
      3.  This is a good place to perform transforms, if you so wish.
          Performing them on batches will speed the process up a bit.
      4.  Return batch of features, labels, lenghts of features,
          and lengths of labels.
      '''
      # batch of input mfcc coefficients
      batch_mfcc = batch # TODO

      # HINT: CHECK OUT -> pad_sequence (imported above)
      # Also be sure to check the input format (batch_first)
      batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True) # TODO
      lengths_mfcc = [len(i) for i in batch_mfcc] # TODO

      # You may apply some transformation, Time and Frequency masking, here in the collate function;
      # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
      #                  -> Would we apply transformation on the validation set as well?
      #                  -> Is the order of axes / dimensions as expected for the transform functions?

      # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
      return batch_mfcc_pad, torch.tensor(lengths_mfcc)


### Config - Hyperparameters

In [13]:
root = '/kaggle/working/11-785-f23-hw3p2'

# # Feel free to add more items here
# config = {
#     "beam_width" : 2,
#     "lr"         : 2e-3,
#     "epochs"     : 50,
#     "batch_size" : 64  # Increase if your device can handle it
# }

config ={
    'epochs'            : 10,
    'batch_size'        : 32,   # Increase if your device can handle it
    'lr'                : 0.0007,
    'encoder dropout'   : 0.25,
    'decoder dropout'   : 0.15,
    'lstm dropout'      : 0.25,
    'factor'            : 0.5,
    'sc_mode'           : 'min',
    'patience'          : 3,
    'sc_threshold'      : 1e-3,
    'threshold mode'    : 'rel',
    'beam_width'        : 5 ,
    'test_beam_width'   : 2
}

# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation
transforms = [] # set of tranformations

### Data loaders

In [14]:
# get me RAMMM!!!!
import gc
gc.collect()

0

In [15]:
# Create objects for the dataset class
train_data = AudioDataset(root_dir = root, partition='train-clean-100', transforms=[], limit=None,train_460=True) #TODO
val_data = AudioDataset(root_dir= root, partition='dev-clean', transforms=[], limit=None, train_460 = False ) # TODO : You can either use the same class with some modifications or make a new one :)
test_data = AudioDatasetTest(root_dir=root, partition='test-clean', transforms=[], limit=None) #TODO

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(dataset = train_data, num_workers=7, batch_size =config['batch_size'],pin_memory=True, shuffle=True, collate_fn= train_data.collate_fn)#TODO
val_loader = torch.utils.data.DataLoader(dataset = val_data, num_workers=4, batch_size =config['batch_size'],pin_memory=True, shuffle=True, collate_fn= val_data.collate_fn)#TODO#TODO
test_loader = torch.utils.data.DataLoader(dataset = test_data, num_workers=1, batch_size =config['batch_size'],pin_memory=True, shuffle=False, collate_fn= test_data.collate_fn)#TODO#TODO

# # Create objects for the dataset class
# train_data = AudioDataset() #TODO
# val_data = ___ # TODO : You can either use the same class with some modifications or make a new one :)
# test_data = AudioDatasetTest() #TODO

# # Do NOT forget to pass in the collate function as parameter while creating the dataloader
# train_loader = #TODO
# val_loader = #TODO
# test_loader = #TODO

print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

before train 360 self.mfccs len = 28539
after train 360 self.mfccs len = 28539
AUDIODATASET LOADED ...
before train 360 self.mfccs len = 2703
after train 360 self.mfccs len = 2703
AUDIODATASET LOADED ...
AUDIODATASET TEST LOADED ...
Batch size:  32
Train dataset samples = 28539, batches = 892
Val dataset samples = 2703, batches = 85
Test dataset samples = 2620, batches = 82


In [16]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break

torch.Size([32, 1630, 28]) torch.Size([32, 203]) torch.Size([32]) torch.Size([32])


# NETWORK

## Basic

This is a basic block for understanding, you can skip this and move to pBLSTM one

In [17]:
torch.cuda.empty_cache()

class Network(nn.Module):

    def __init__(self, hidden_size, nlayers, out_size=47, embed_size=40):

        super(Network, self).__init__()
        self.nlayers = nlayers
        self.hidden_size = hidden_size
        self.embed_size = embed_size
        self.out_size = out_size
        self.cnns = torch.nn.Sequential(
            nn.Conv1d(self.embed_size, self.hidden_size, 3, padding=1, bias=False),
            nn.BatchNorm1d(self.hidden_size),
            nn.ReLU(inplace=True))
        # Adding some sort of embedding layer or feature extractor might help performance.
        # self.embedding = ?

        # TODO : look up the documentation. You might need to pass some additional parameters.
        self.lstm = nn.LSTM(input_size=self.hidden_size,
                            hidden_size=self.hidden_size,
                            num_layers=3,
                            bias=True,
                            batch_first=True,
                            dropout=0.2, # regularization
                            bidirectional=True)

        self.classification = nn.Sequential(
            #TODO: Linear layer with in_features from the lstm module above and out_features = OUT_SIZE
            nn.Linear(self.hidden_size*2, self.hidden_size),
            nn.Linear(self.hidden_size, self.out_size)

        )


        # self.logSoftmax = nn.log_softmax(2)#TODO: Apply a log softmax here. Which dimension would apply it on ?
        self.logSoftmax =  nn.LogSoftmax(2)

    def forward(self, x, lx):
        #TODO
        # The forward function takes 2 parameter inputs here. Why?
        # Refer to the handout for hints
        x_cnn_input = x.permute(0, 2, 1) # (B, C_in, T_in)
        # x_cnn_input = x
        # x_cnn_input = x.transpose(1, 2)
        x_post_cnn = self.cnns(x_cnn_input) # (B, C_out, T_out)
        x_rnn_in = x_post_cnn.permute(2, 0, 1) # (T, B, C_out)
        # x_rnn_in = x_post_cnn
        # x_rnn_in = x_post_cnn.transpose(1, 2)
        x_packed = pack_padded_sequence(x_rnn_in, lx, enforce_sorted=False)
        out_packed, hidden = self.lstm(x_packed)
        out, out_lens = pad_packed_sequence(out_packed, batch_first=True) # (B, T, C)

        # Log softmax after output layer is required since nn.CTCLoss expect log prob
        out_prob = self.classification(out) # (B, T, Classes=47)
        out_prob = self.logSoftmax(out_prob)

        # Permute to fit for input format of CTCLoss
        # out_prob = out_prob.permute(1, 0, 2) #torch.transpose(out_prob, 0, 1) # (T, B, C)

        # TODO: calculate new xLens
        return out_prob, lx

## Initialize Basic Network
(If trying out the basic Network)

In [18]:
torch.cuda.empty_cache()

# model = Network().to(device)
# summary(model, x.to(device), lx) # x and lx come from the sanity check above :)
model = Network(hidden_size=256, nlayers=3, out_size=len(PHONEMES), embed_size=28).to(device)
print(model)
# summary(model, x.to(device), lx)

Network(
  (cnns): Sequential(
    (0): Conv1d(28, 256, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (lstm): LSTM(256, 256, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
  (classification): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=41, bias=True)
  )
  (logSoftmax): LogSoftmax(dim=2)
)


## ASR Network

### Pyramid Bi-LSTM (pBLSTM)

In [19]:
# Utils for network
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

In [20]:
class pBLSTM(torch.nn.Module):

    '''
    Pyramidal BiLSTM
    Read the write up/paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input?
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''

    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()

        self.blstm = nn.LSTM(input_size=2*input_size,hidden_size=hidden_size,num_layers=1,batch_first=True,bidirectional=True)# TODO: Initialize a single layer bidirectional LSTM with the given input_size and hidden_size

    def forward(self, x_packed): # x_packed is a PackedSequence

        # TODO: Pad Packed Sequence

        # Call self.trunc_reshape() which downsamples the time steps of x and increases the feature dimensions as mentioned above
        # self.trunc_reshape will return 2 outputs. What are they? Think about what quantites are changing.
        # TODO: Pack Padded Sequence. What output(s) would you get?
        # TODO: Pass the sequence through bLSTM

        x_pad, x_seq_len = pad_packed_sequence(x_packed, batch_first= True)
        x_trunc, x_len = self.trunc_reshape(x_pad, x_seq_len)
        x_packed = pack_padded_sequence(x_trunc, x_len, batch_first= True, enforce_sorted= False)
        output, (h_n, c_n) = self.blstm(x_packed)

        # What do you return?

        return output

    def trunc_reshape(self, x, x_lens):
        # TODO: If you have odd number of timesteps, how can you handle it? (Hint: You can exclude them)
        # TODO: Reshape x. When reshaping x, you have to reduce number of timesteps by a downsampling factor while increasing number of features by the same factor
        # TODO: Reduce lengths by the same downsampling factor

        if x.shape[1]%2 != 0:
          x = x[:,:-1,:]
          x_lens -= 1
        x = x.reshape((x.shape[0],x.shape[1]//2, x.shape[2]*2))
        x_lens = x_lens//2
        return x, x_lens

In [21]:
from torch.autograd import Variable
class LockedDropout(nn.Module):
  def __init__(self, dropout):
    super().__init__()
    self.dropout = dropout

  def forward(self, x):
    if not self.training or not self.dropout:
      return x
    x_unpacked, x_lens = pad_packed_sequence(x, batch_first= True)
    m = x.data.new(1, x_unpacked.size(1), x_unpacked.size(2)).bernoulli_(1 - self.dropout)
    mask = Variable(m, requires_grad= False) / (1 - self.dropout)
    mask = mask.expand_as(x_unpacked)
    out = mask * x_unpacked
    return pack_padded_sequence(out, lengths= x_lens, batch_first= True, enforce_sorted= False)

### Encoder

In [22]:
class Encoder(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, input_size, encoder_hidden_size):
        super(Encoder, self).__init__()


        self.embedding = torch.nn.Sequential(
            PermuteBlock(),
            torch.nn.Conv1d(in_channels= 28, out_channels= 28, kernel_size= 7, padding= 3),
            torch.nn.GELU(),
            PermuteBlock()
        )#TODO: You can use CNNs as Embedding layer to extract features. Keep in mind the Input dimensions and expected dimension of Pytorch CNN.
        self.BLSTMs = torch.nn.LSTM(input_size=28, hidden_size= encoder_hidden_size, num_layers=3, batch_first= True, bidirectional= True, dropout= config['lstm dropout'])

        self.pBLSTMs = torch.nn.Sequential( # How many pBLSTMs are required?
            # TODO: Fill this up with pBLSTMs - What should the input_size be?
            # Hint: You are downsampling timesteps by a factor of 2, upsampling features by a factor of 2 and the LSTM is bidirectional)
            # Optional: Dropout/Locked Dropout after each pBLSTM (Not needed for early submission)
            # https://github.com/salesforce/awd-lstm-lm/blob/dfd3cb0235d2caf2847a4d53e1cbd495b781b5d2/locked_dropout.py#L5
            pBLSTM(encoder_hidden_size * 2, encoder_hidden_size),
            LockedDropout(config['encoder dropout']),
            pBLSTM(encoder_hidden_size * 2, encoder_hidden_size),
            LockedDropout(config['encoder dropout'])
        )

    def forward(self, x, x_lens):
        # Where are x and x_lens coming from? The dataloader
        #TODO: Call the embedding layer
        # TODO: Pack Padded Sequence
        # TODO: Pass Sequence through the pyramidal Bi-LSTM layer
        # TODO: Pad Packed Sequence
        x_out = self.embedding(x)
        x_out = pack_padded_sequence(x_out, x_lens, batch_first= True, enforce_sorted= False)
        x_blstm, (h_n, c_n) = self.BLSTMs(x_out)
        x_pblstm = self.pBLSTMs(x_blstm)
        encoder_outputs, encoder_lens = pad_packed_sequence(x_pblstm, batch_first= True)
        # Remember the number of output(s) each function returns

        return encoder_outputs, encoder_lens

### Decoder

In [23]:
class Decoder(torch.nn.Module):

    def __init__(self, embed_size, output_size= 41):
        super().__init__()

        self.mlp = torch.nn.Sequential(
            PermuteBlock(), torch.nn.BatchNorm1d(2*embed_size), PermuteBlock(),
            #TODO define your MLP arch. Refer HW1P2
            #Use Permute Block before and after BatchNorm1d() to match the size
            torch.nn.Linear(2 * embed_size, 2 * embed_size),
            torch.nn.GELU(),
            torch.nn.Dropout(config['decoder dropout']),
            torch.nn.Linear(2 * embed_size, 2 * embed_size),
            torch.nn.GELU(),
            torch.nn.Dropout(config['decoder dropout']),
            torch.nn.Linear(2 * embed_size, output_size)
            )

        self.softmax = torch.nn.LogSoftmax(dim=2)

    def forward(self, encoder_out):
        #TODO call your MLP
        #TODO Think what should be the final output of the decoder for the classification
        mlp_out = self.mlp(encoder_out)
        out = self.softmax(mlp_out)
        return out

In [24]:
class ASRModel(torch.nn.Module):

    def __init__(self, input_size, embed_size= 192, output_size= len(PHONEMES)):
        super().__init__()

        self.augmentations  = torch.nn.Sequential(
            #TODO Add Time Masking/ Frequency Masking
            #Hint: See how to use PermuteBlock() function defined above
        )
        self.encoder        = Encoder(input_size, embed_size)# TODO: Initialize Encoder
        self.decoder        = Decoder(embed_size, output_size)# TODO: Initialize Decoder



    def forward(self, x, lengths_x):

        if self.training:
            x = self.augmentations(x)

        encoder_out, encoder_lens   = self.encoder(x, lengths_x)
        decoder_out                 = self.decoder(encoder_out)

        return decoder_out, encoder_lens

## Initialize ASR Network

In [25]:
# model = ASRModel(
#     input_size  = #TODO,
#     embed_size  = #TODO
#     output_size = len(PHONEMES)
# ).to(device)
# print(model)
# summary(model, x.to(device), lx)
model = ASRModel(
    input_size  = 28,
    embed_size  = 256,#TODO
    output_size = len(PHONEMES)
).to(device)
print(model)
#summary(model, x.to(device), lx)

ASRModel(
  (augmentations): Sequential()
  (encoder): Encoder(
    (embedding): Sequential(
      (0): PermuteBlock()
      (1): Conv1d(28, 28, kernel_size=(7,), stride=(1,), padding=(3,))
      (2): GELU(approximate='none')
      (3): PermuteBlock()
    )
    (BLSTMs): LSTM(28, 256, num_layers=3, batch_first=True, dropout=0.25, bidirectional=True)
    (pBLSTMs): Sequential(
      (0): pBLSTM(
        (blstm): LSTM(1024, 256, batch_first=True, bidirectional=True)
      )
      (1): LockedDropout()
      (2): pBLSTM(
        (blstm): LSTM(1024, 256, batch_first=True, bidirectional=True)
      )
      (3): LockedDropout()
    )
  )
  (decoder): Decoder(
    (mlp): Sequential(
      (0): PermuteBlock()
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PermuteBlock()
      (3): Linear(in_features=512, out_features=512, bias=True)
      (4): GELU(approximate='none')
      (5): Dropout(p=0.15, inplace=False)
      (6): Linear(in_features=

In [26]:
checkpoint = torch.load("/kaggle/input/checkcheckers/best(2).pth") # change the path to your actual path
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

# Training Config
Initialize Loss Criterion, Optimizer, CTC Beam Decoder, Scheduler, Scaler (Mixed-Precision), etc.

In [27]:
criterion = nn.CTCLoss(blank= CMUdict.index(''))# Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

# optimizer =  torch.optim.AdamW(...) # What goes in here?
optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(labels=LABELS, beam_width=config['beam_width'], num_processes= 4, log_probs_input=True)#TODO

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode= config['sc_mode'], factor= config['factor'], patience= config['patience'], threshold= config['sc_threshold'], threshold_mode= config['threshold mode'],  verbose=True)#TODO

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

# Decode Prediction

In [28]:
# def decode_prediction(output, output_lens, decoder, PHONEME_MAP= LABELS):

#     # TODO: look at docs for CTC.decoder and find out what is returned here. Check the shape of output and expected shape in decode.
#     (...) = decoder.decode(output, seq_lens= output_lens) #lengths - list of lengths

#     pred_strings                    = []

#     for i in range(output_lens.shape[0]):
#         #TODO: Create the prediction from the output of decoder.decode. Don't forget to map it using PHONEMES_MAP.

#     return pred_strings

# def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP= LABELS): # y - sequence of integers

#     dist            = 0
#     batch_size      = label.shape[0]

#     pred_strings    = decode_prediction(output, output_lens, decoder, PHONEME_MAP)

#     for i in range(batch_size):
#         # TODO: Get predicted string and label string for each element in the batch
#         pred_string = #TODO
#         label_string = #TODO
#         dist += Levenshtein.distance(pred_string, label_string)

#     #dist /= batch_size # TODO: Uncomment this, but think about why we are doing this
#     raise NotImplemented
#     # return dist

def decode_prediction(output, output_lens, decoder, PHONEME_MAP= LABELS):

    # TODO: look at docs for CTC.decoder and find out what is returned here. Check the shape of output and expected shape in decode.
    output = torch.transpose(output, 0, 1)
    output, _, _, out_seq_len = decoder.decode(output, seq_lens= output_lens) #lengths - list of lengths

    pred_strings = []

    for i in range(output_lens.shape[0]):
        #TODO: Create the prediction from the output of decoder.decode. Don't forget to map it using PHONEMES_MAP.
        logits=output[i][0][:out_seq_len[i][0]]
        # print('index',i)
        # print('logits',max(logits))
        try: pred_strings.append(''.join([PHONEME_MAP[i] for i in logits]))
        except:
          print(logits)
          return
    return pred_strings

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP= LABELS): # y - sequence of integers

    dist            = 0
    batch_size      = label.shape[0]

    pred_strings    = decode_prediction(output, output_lens, decoder, PHONEME_MAP)
    # print(pred_strings)
    for i in range(batch_size):
        # TODO: Get predicted string and label string for each element in the batch
        pred_string = pred_strings[i]#TODO
        label_string = ''.join(PHONEME_MAP[l] for l in label[i][:label_lens[i]])#TODO
        dist += Levenshtein.distance(pred_string, label_string)

    dist /= batch_size # TODO: Uncomment this, but think about why we are doing this
    return dist

# Test Implementation

In [29]:
# test code to check shapes

model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)

    print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))

    break

torch.Size([32, 406, 41])
torch.Size([406, 32, 41]) torch.Size([32, 181])
tensor(0.3836, device='cuda:0', grad_fn=<MeanBackward0>)
17.1875


# WandB

You will need to fetch your api key from wandb.ai

In [30]:
import wandb
wandb.login(key="f18fc83a91cadf0882193f06fada009a25702e01") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [31]:
run = wandb.init(
    name = "early-submission", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
#     id ='run062', ### Insert specific run id here if you want to resume a previous run
#     resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

wandb: Currently logged in as: dkwizera. Use `wandb login --relogin` to force relogin


In [32]:
# api = wandb.Api()
# run_downloader = api.run("djpr/hw2p2-ablations_final/evg28hz7")
# run_downloader.file("checkpoint.pth").download()

# Train Functions

In [33]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):

    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16.
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar

    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(h, y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss,val_dist

## Training Setup

In [34]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [35]:
# This is for checkpointing, if you're doing it over multiple sessions
last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
epoch_model_path = "/kaggle/working/check.pth"#TODO set the model path( Optional, you can just store best one. Make sure to make the changes below )
best_model_path = "/kaggle/working/best.pth"#TODO set best model path
# last_epoch_completed = 0
# start = last_epoch_completed
# end = config["epochs"]
# best_lev_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
# epoch_model_path = #TODO set the model path( Optional, you can just store best one. Make sure to make the changes below )
# best_model_path = #TODO set best model path

In [36]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    curr_lr = float(optimizer.param_groups[0]['lr']) #TODO

    train_loss              = train_model(model, train_loader, criterion, optimizer)#TODO
    valid_loss, valid_dist  = validate_model(model, val_loader, decoder, phoneme_map= LABELS)#TODO
    scheduler.step(valid_dist)

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}%\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    wandb.log({
        'train_loss': train_loss,
        'valid_dist': valid_dist,
        'valid_loss': valid_loss,
        'lr'        : curr_lr
    })

    save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
    wandb.save(epoch_model_path)
    print("Saved epoch model")

    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
        wandb.save(best_model_path)
        print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()


Epoch: 1/10


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


	Train Loss 0.0836	 Learning Rate 0.0007000
	Val Dist 5.4641%	 Val Loss 0.3399
Saved epoch model
Saved best model

Epoch: 2/10


	Train Loss 0.0793	 Learning Rate 0.0007000
	Val Dist 5.4709%	 Val Loss 0.3444
Saved epoch model

Epoch: 3/10


	Train Loss 0.0742	 Learning Rate 0.0007000
	Val Dist 5.4451%	 Val Loss 0.3491
Saved epoch model
Saved best model

Epoch: 4/10


...

	Train Loss 0.0658	 Learning Rate 0.0007000
	Val Dist 5.5056%	 Val Loss 0.3672
Saved epoch model

Epoch: 8/10


	Train Loss 0.0668	 Learning Rate 0.0007000
	Val Dist 5.5150%	 Val Loss 0.3587
Saved epoch model

Epoch: 9/10


	Train Loss 0.0633	 Learning Rate 0.0007000
	Val Dist 5.3326%	 Val Loss 0.3618
Saved epoch model
Saved best model

Epoch: 10/10


	Train Loss 0.0623	 Learning Rate 0.0007000
	Val Dist 5.5335%	 Val Loss 0.3744
Saved epoch model


lr,▁▁▁▁▁▁▁▁▁▁
train_loss,█▇▅▄▃▃▂▃▁▁
valid_dist,▆▆▅▃▅▃▇▇▁█
valid_loss,▁▂▃▄▄▄▇▅▅█
lr,0.0007
train_loss,0.06226
valid_dist,5.53348
valid_loss,0.37436


# Generate Predictions and Submit to Kaggle

In [38]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

# TEST_BEAM_WIDTH = #TODO

# test_decoder    = #TODO
# results = []

# model.eval()
# print("Testing")
# for data in tqdm(test_loader):

#     x, lx   = data
#     x       = x.to(device)

#     with torch.no_grad():
#         h, lh = model(x, lx)

#     prediction_string= # TODO call decode_prediction
#     #TODO save the output in results array.

#     del x, lx, h, lh
#     torch.cuda.empty_cache()

TEST_BEAM_WIDTH = 20 #TODO

test_decoder    = CTCBeamDecoder(labels=LABELS, beam_width=TEST_BEAM_WIDTH, log_probs_input=True)#TODO
results = []

model.eval()
print("Testing")
for data in tqdm(test_loader):

    x, lx   = data
    x       = x.to(device)

    with torch.no_grad():
        h, lh = model(x, lx)
        h = torch.permute(h, (1, 0, 2))
    prediction_string= decode_prediction(h, lh, test_decoder, LABELS)# TODO call decode_prediction
    #TODO save the output in results array.
    results.extend(prediction_string)

    del x, lx, h, lh
    torch.cuda.empty_cache()

Testing


100%|██████████| 82/82 [00:46<00:00,  1.76it/s]


In [39]:
data_dir = f"{root}/test-clean/random_submission.csv"
df = pd.read_csv(data_dir)
df.label = results
df.to_csv('submission.csv', index = False)

In [40]:
!kaggle competitions submit -c automatic-speech-recognition-asr -f submission.csv -m "I made it!"

100%|█████████████████████████████████████████| 209k/209k [00:01<00:00, 111kB/s]
Successfully submitted to Automatic Speech Recognition (ASR)